In [4]:
import torch.nn as nn 
class BILSTM(nn.Module):
  def __init__(self,):
    super(BILSTM,self).__init__()
    
      
      
    
  

In [ ]:
import torch 
import torch.nn as nn 
class sequenceclassifier(nn.Module):   #nn is class in pytorch  #use `PascalCase` for class names in Python.
  #defining the class fors sequence classification using a bidirectional LSTM
  def __init__(self,vocab_size,embedding_dim,hidden_dim,num_layers,dropout,output_dim):      # ??constructor to initialize the parameters
    super(sequenceclassifier,self).__init__()
    self.embedding = nn.Embedding(vocab_size,embedding_dim)
    self.lstm      = nn.LSTM(embedding_dim,
                             hidden_dim,
                             num_layers,
                             batch_first=True,#!by default, the input shape of lstm is sequence_length,batch_size,embedding_dim, but we want batch_size first as input shape is batch_size,sequence_length
                             bidirectional = True,
                             dropout       = dropout)
    #Pytorch is case sensetive. All module names uses upper case letter.
    #like nn.LSTM,nn.RNN
    #input dimension= embedding_dim
    #both ht and ct are of same size(num_layers*num_directions,batch_size,hidden_dim) 
    #number of direction is 2 for bidirectional lstm 
  
    #number of layers=num_layers is the number of stacked LSTM layers (vertical depth).
    #make sure layers are capitalized while calling, we will not use small letter like nn.LSTM or nn.Linear not nn.linear 
    self.dropout=nn.Dropout(dropout)
    self.fc=nn.Linear(hidden_dim *2,output_dim)
  def forward(self,x): # !size of x is (batch_Size,sequence_length)
    #forward pass where actual calculation/computation happens.
    embedded=self.embedding(x)                               #  self.lstm(embedded) returns a tuple: (output, (h_n, c_n))
    output,(h_n,c_n)=self.lstm(embedded)
    hidden=torch.cat((h_n[-2],h_n[-1]),dim=1) #hidden after concatentaion : (batch_size,hidden_dim *2)
    
    '''  for sequence labeling, we use output as input for next layer
         for sequence classification, we use the concatenated hidden state as input for next layer
         
         1.h_n[-2] → last forward layer

         2.h_n[-1] → last backward layer

         '''
    dropout = self.dropout(hidden) #after dropout, (batch_size,hidden_dim *2)
    out     = self.fc(dropout) #after linear layer, (batch_size,output_dim)
    return out                                                                      
    
    

: 

: 

: 


**Summary of Tensor Shapes in the Model:**

- **Input x:** Shape is (batch_size, sequence_length)
- **After Embedding Layer:** Shape is (batch_size, sequence_length, embedding_dim)
- **After LSTM Layer:**
  - Output: Shape is (batch_size, sequence_length, hidden_dim * 2) (since it’s bidirectional)
  - h_n: Shape is (num_layers * 2, batch_size, hidden_dim)
- **After Concatenating Last Layer’s Forward and Backward h_n:** Shape is (batch_size, hidden_dim * 2)
- **After Dropout:** Shape remains (batch_size, hidden_dim * 2)
- **After Linear Layer:** Shape is (batch_size, output_dim)

---




In [ ]:
import torch
import torch.nn as nn 
class encoder(nn.Module): 
  def __init__(self,embedding_dim,hidden_dim,num_layers,dropout,): 
    super(encoder,self).__init__()
    self.embedding = nn.Embedding(embedding_dim,hidden_dim)
    self.lstm=nn.LSTM(embedding_dim,
                      hidden_dim,
                      num_layers,
                      batch_first = True,
                      bidirectional=True)
    self.dropout = nn.Dropout(dropout)
    self.fc      = nn.Linear(hidden_dim *2,hidden_dim) #hidden dim
    
  def forward(self,x): #!dim of x is (batch_size,seqence_lenght)
    embedded = self.embedding(x)
    output,(h_n,c_n)     = self.lstm(embedded)  #lstm returns a tuple:(output,(h_n,c_n))
    #output is output from each time step
    #h_n and c_n are hidden and cell state of the last time step.
    hidden  = torch.cat((h_n[-2],h_n[-1]),dim=1)
    dropout = self.Dropout(hidden)
    out     = self.fc(dropout)
    return out 
    
    
    
    


: 

: 

: 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AdditiveAttention(nn.Module):
    """
    Implements Bahdanau (additive) attention.
    """
    def __init__(self, encoder_hidden_dim, decoder_hidden_dim, attention_dim):
        super(AdditiveAttention, self).__init__()
        self.encoder_attn = nn.Linear(encoder_hidden_dim, attention_dim)
        self.decoder_attn = nn.Linear(decoder_hidden_dim, attention_dim)
        self.v = nn.Parameter(torch.rand(attention_dim))  #self.v: shape (attn_dim,) or (attn_dim, 1)

    def forward(self, encoder_outputs, decoder_hidden):
        # encoder_outputs: (batch, src_len, encoder_hidden_dim)
        # decoder_hidden: (batch, decoder_hidden_dim)
        batch_size, src_len, _ = encoder_outputs.size()
        
        # Repeat decoder hidden state src_len times
        decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        # Calculate energy
        energy = torch.tanh(self.encoder_attn(encoder_outputs) + self.decoder_attn(decoder_hidden)) # (batch, src_len, attn_dim)
        #!(batch, src_len, attn_dim) @ (attn_dim,) = batch,src_len 
        energy = energy @ self.v    # (batch, src_len)
        
        attn_weights = F.softmax(energy, dim=1) # (batch, src_len)
        
        # Compute context vector
                   context              = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1) # (batch, encoder_hidden_dim)
        #dimension of attn_weights      = batch, src_len,
        #          #dimension of encoder_output = batch, src_len, encoder_hidden_dim
        
        return context, attn_weights

In [ ]:
import torch
import torch.nn as nn
class KvAttention(nn.Module): 
  def __init__(self,encoder_hidden_dim,decoder_hidden_dim,attention_dim,):
    super(KvAttention,self).__init__()
    self.encoder_attn = nn.Linear(encoder_hidden_dim,attention_dim) #linear layer to make similar dimension of encoder hidden size and decoder hidden size
    
    self.decoder_attn = nn.Linear(decoder_hidden_dim,attention_dim)
    self.v            = nn.Parameter(torch.rand(attention_dim))
  def forward(self,encoder_outputs,decoder_hidden): 
      #encoder_outputs                            : (``batch,src_len,encoder_hidden_dim``)
      #decoder_hidden                             : shape(`batch_size ,decoder_hidden_dim`)
    batch_size,src_len, _ = encoder_outputs.size() #get the batch size and source length from encoder outputs
    #- means we ignore the third dimension
    #batch_size is the number of samples in one batch (say 64 sentences)
    #src_len is the length of the source sequence (say 50 words)
    decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1,src_len,1).repeat((1,src_len,1))
    #repeat the decoder hidden state src_len times to match the source length
    #unsqueeze(1) adds a new dimension at index 1, making it  batch_size, 1, decoder_hidden_dim
    
    
    
    
    
    
    

: 

: 

: 

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DotProductAttention(nn.Module):
    def __init__(self):
        super(DotProductAttention, self).__init__()

    def forward(self, encoder_outputs, decoder_hidden):
        """
        encoder_outputs: (batch, src_len, hidden_dim)
        decoder_hidden: (batch, hidden_dim)
        """
        # Unsqueeze to (batch, 1, hidden_dim) for bmm
        decoder_hidden = decoder_hidden.unsqueeze(1)  # (batch, 1, hidden_dim)

        # Compute dot product attention scores
        attn_scores = torch.bmm(decoder_hidden, encoder_outputs.transpose(1, 2))  # (batch, 1, src_len)
        
        # Normalize scores to get attention weights
        attn_weights = F.softmax(attn_scores, dim=-1)  # (batch, 1, src_len)

        # Weighted sum of encoder outputs to get context
        context = torch.bmm(attn_weights, encoder_outputs)  # (batch, 1, hidden_dim)

        # Remove extra dimension
        context = context.squeeze(1)       # (batch, hidden_dim)
        attn_weights = attn_weights.squeeze(1)  # (batch, src_len)

        return context, attn_weights
